## Create Decision Tree Model to Forecast Returns

#### Import Packages

In [9]:
import pandas as pd
import datetime
import gc
from sklearn.ensemble import (
BaggingRegressor, RandomForestRegressor, AdaBoostRegressor)
from sklearn.metrics import mean_squared_error
from technical_indicators import * # import all function
from sklearn.model_selection import TimeSeriesSplit
#import parfit as pf
from sklearn.metrics import r2_score
import numpy as np
from sklearn.model_selection import ParameterGrid
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from sklearn import linear_model

#### Read in Data via GitHub URL

In [10]:
url = "https://raw.githubusercontent.com/meenmo/Stat479_Project/master/Data/IBM.csv"
df_ORIGINAL = pd.read_csv(url)


***
## Clean Data & Create Technical Indicator Variables

- Create Deep copy of dataframe
- Use Adjusted Close Data
- Drop Close 
- Rename "Adj. Close" as "Close"

In [43]:
df_features = df_ORIGINAL.copy(deep=True) # Create Deep
df_features.drop(['Close'], axis = 1, inplace = True) # drop close column
df_features.columns = ['Date', 'High', 'Low', 'Open', 'Volume', 'Close'] # Close is actually Adj. Close

df_features['Date'] = pd.to_datetime(df_features['Date'])
df_features.head() # sanity check


Date        High         Low        Open   Volume      Close
0 2002-01-02  121.500000  119.800003  120.599999  6862800  84.677422
1 2002-01-03  124.220001  120.250000  121.500000  8621700  86.182800
2 2002-01-04  125.599999  123.980003  124.050003  8405200  87.534859
3 2002-01-07  126.190002  123.699997  125.000000  5939600  86.454575
4 2002-01-08  125.199997  123.730003  124.250000  5311800  86.907600

#### Function: Create Lagged Returns

In [44]:
"""
Creates Lagged Returns 
- given OHLCV dataframe
- numer of lagged days
"""
def create_lag_features(df, lag_days):
    df_ret = df.copy()
    
    # iterate through the lag days to generate lag values up to lag_days + 1
    for i in range(1,lag_days + 2):
        df_lag = df_ret[['Date', 'Close']].copy()
        # generate dataframe to shift index by i day.
        df_lag['Date'] = df_lag['Date'].shift(-i)
        df_lag.columns = ['Date', 'value_lag' + str(i)]
        # combine the valuelag
        df_ret = pd.merge(df_ret, df_lag, how = 'left', left_on = ['Date'], right_on = ['Date'])
    
    #frees memory
    del df_lag
    
    # calculate today's percentage lag
    df_ret['Today'] = (df_ret['Close'] - df_ret['value_lag1'])/(df_ret['value_lag1']) * 100.0 
    
    # calculate percentage lag
    for i in range(1, lag_days + 1):
        df_ret['lag' + str(i)] = (df_ret['value_lag'+ str(i)] - df_ret['value_lag'+ str(i+1)])/(df_ret['value_lag'+str(i+1)]) * 100.0
    
    # drop unneeded columns which are value_lags
    for i in range(1, lag_days + 2):
        df_ret.drop(['value_lag' + str(i)], axis = 1, inplace = True)
                                                                                                                                                                                                                                                                                                                                     
    return df_ret


### Run Function
df_features = create_lag_features(df_features, 5) # 5 lag features
df_features.head(7)

Date        High         Low        Open   Volume      Close  \
0 2002-01-02  121.500000  119.800003  120.599999  6862800  84.677422   
1 2002-01-03  124.220001  120.250000  121.500000  8621700  86.182800   
2 2002-01-04  125.599999  123.980003  124.050003  8405200  87.534859   
3 2002-01-07  126.190002  123.699997  125.000000  5939600  86.454575   
4 2002-01-08  125.199997  123.730003  124.250000  5311800  86.907600   
5 2002-01-09  126.389999  124.150001  124.699997  6839900  86.761269   
6 2002-01-10  124.000000  121.419998  123.750000  8926300  85.123428   

      Today      lag1      lag2      lag3      lag4      lag5  
0       NaN       NaN       NaN       NaN       NaN       NaN  
1  1.777781       NaN       NaN       NaN       NaN       NaN  
2  1.568826  1.777781       NaN       NaN       NaN       NaN  
3 -1.234119  1.568826  1.777781       NaN       NaN       NaN  
4  0.524004 -1.234119  1.568826  1.777781       NaN       NaN  
5 -0.168376  0.524004 -1.234119  1.568826  1.777781       NaN  
6 -1.887755 -0.168376  0.524004 -1.234119  1.568826  1.777781

#### Drop Rows with NaN

In [45]:
# drop earlier data with missing lag features
df_features.dropna(inplace=True)
# reset index
df_features.reset_index(drop = True, inplace = True)


#### Create Technical Indicators

- Create Technical Indicators
- Call functions from `technical_inicators.py`
- Drop Rows with NaN
  

In [46]:

#### GENERATE TECHNICAL INDICATORS FEATURES
df_features = standard_deviation(df_features, 14)

df_features = relative_strength_index(df_features, 14) # periods
df_features = average_directional_movement_index(df_features, 14, 13) # n, n_ADX
df_features = moving_average(df_features, 21) # periods
df_features = exponential_moving_average(df_features, 21) # periods
df_features = momentum(df_features, 14) # 

df_features = average_true_range(df_features, 14)
df_features = bollinger_bands(df_features, 21)
df_features = ppsr(df_features)
df_features = stochastic_oscillator_k(df_features)
df_features = stochastic_oscillator_d(df_features, 14)
df_features = trix(df_features, 14)
df_features = macd(df_features, 26, 12)
df_features = mass_index(df_features)
df_features = vortex_indicator(df_features, 14)

df_features = kst_oscillator(df_features, 10, 10, 10, 15, 10, 15, 20, 30)
df_features = true_strength_index(df_features, 25, 13)

#df_features = accumulation_distribution(df_features, 14) # Causes Problems, apparently
df_features = chaikin_oscillator(df_features)
df_features = money_flow_index(df_features, 14)
df_features = on_balance_volume(df_features, 14)
df_features = force_index(df_features, 14)
df_features = ease_of_movement(df_features, 14)
df_features = commodity_channel_index(df_features, 14)
df_features = keltner_channel(df_features, 14)
df_features = ultimate_oscillator(df_features)
df_features = donchian_channel(df_features, 14)
    
# drop earlier data with missing lag features
df_features.dropna(inplace=True)
df_features = df_features.reset_index(drop = True)


In [47]:
### Sanity Check
df_features.head(10)

Date        High         Low        Open    Volume      Close  \
0 2002-03-14  107.949997  106.589996  107.019997   5335500  74.391068   
1 2002-03-15  107.449997  105.589996  106.550003  10864100  74.523666   
2 2002-03-18  108.639999  106.230003  107.099999   5301200  74.216637   
3 2002-03-19  108.050003  106.489998  106.849999   4614800  75.012154   
4 2002-03-20  106.900001  105.489998  106.900001   4844100  73.623466   
5 2002-03-21  106.779999  104.699997  105.699997   5113100  74.516685   
6 2002-03-22  106.699997  105.070000  106.500000   5507900  73.693192   
7 2002-03-25  106.660004  103.500000  105.800003   6277900  72.269615   
8 2002-03-26  105.699997  102.300003  103.570000   8144000  71.809052   
9 2002-03-27  103.790001  102.500000  103.750000   5602400  72.150978   

      Today      lag1      lag2      lag3     ...         MFI_14  \
0 -0.541176 -1.216580  3.097695  0.142742     ...       0.714286   
1  0.178246 -0.541176 -1.216580  3.097695     ...       0.642857   
2 -0.411990  0.178246 -0.541176 -1.216580     ...       0.714286   
3  1.071885 -0.411990  0.178246 -0.541176     ...       0.714286   
4 -1.851283  1.071885 -0.411990  0.178246     ...       0.642857   
5  1.213226 -1.851283  1.071885 -0.411990     ...       0.571429   
6 -1.105114  1.213226 -1.851283  1.071885     ...       0.500000   
7 -1.931761 -1.105114  1.213226 -1.851283     ...       0.428571   
8 -0.637285 -1.931761 -1.105114  1.213226     ...       0.357143   
9  0.476161 -0.637285 -1.931761 -1.105114     ...       0.357143   

         OBV_14      Force_14        EoM_14    CCI_14  KelChM_14  KelChU_14  \
0  1.962464e+06 -2.921157e+07  2.286739e-07  0.959960  93.032764  95.832764   
1  3.445814e+06  5.695471e+06  2.086708e-07  0.713277  93.570735  96.351449   
2  3.898193e+06 -4.066156e+07  2.500726e-07  0.779588  94.192408  96.916693   
3  3.372200e+06 -4.964177e+07  2.263920e-07  0.762796  94.770295  97.348152   
4  2.459121e+06 -1.593928e+07  2.007584e-07  0.060704  95.249347  97.800775   
5  2.074850e+06 -1.411611e+07  1.220918e-07 -0.248791  95.547059  97.918488   
6  9.123071e+05  1.101343e+06  3.426329e-08 -0.533297  95.592311  97.837311   
7  1.004036e+06  1.890940e+06 -5.730833e-09 -1.582313  95.545348  97.848920   
8 -1.267786e+05 -1.083363e+06 -5.076478e-08 -1.978289  95.397427  97.776713   
9  1.057100e+06  1.198994e+06 -4.957376e-08 -1.969496  95.286158  97.453301   

   KelChD_14  Ultimate_Osc  Donchian_14  
0  90.232765      0.034641    13.389999  
1  90.790020      0.031385    13.389999  
2  91.468123      0.058735    13.389999  
3  92.192438      0.057835    13.389999  
4  92.697918      0.026201    13.389999  
5  93.175630     -0.008129    13.389999  
6  93.347311     -0.021320    13.389999  
7  93.241776     -0.045311    13.389999  
8  93.018141     -0.061006    11.739998  
9  93.119015     -0.040421    11.739998  

[10 rows x 49 columns]

***
## Train-Test Split

#### (Subject to change depending)
-  Train     : 2002 - 2017
    -  Validation: 2015 - 2017
    - OR         : Time Series Validation from the website
-  Test      : 2018 - Oct 31, 2018

#### In this Notebook
- Train : 2002 - 2017
- Test  : 2018 - Oct 31, 2018


#### Note
- y is Todays Daily Return.
- We wan't to PREDICT Tomorrows Daily Return
- Hence, we train the model on data __[start, today]__ and __predict y[today + 1]__ and NOT predict y[today]




In [48]:
df_features['Future Returns'] = df_features['Today'].shift(-1) 

# drop earlier data with missing lag features
df_features.dropna(inplace=True)

df_features.head()


Date        High         Low        Open    Volume      Close  \
0 2002-03-14  107.949997  106.589996  107.019997   5335500  74.391068   
1 2002-03-15  107.449997  105.589996  106.550003  10864100  74.523666   
2 2002-03-18  108.639999  106.230003  107.099999   5301200  74.216637   
3 2002-03-19  108.050003  106.489998  106.849999   4614800  75.012154   
4 2002-03-20  106.900001  105.489998  106.900001   4844100  73.623466   

      Today      lag1      lag2      lag3       ...              OBV_14  \
0 -0.541176 -1.216580  3.097695  0.142742       ...        1.962464e+06   
1  0.178246 -0.541176 -1.216580  3.097695       ...        3.445814e+06   
2 -0.411990  0.178246 -0.541176 -1.216580       ...        3.898193e+06   
3  1.071885 -0.411990  0.178246 -0.541176       ...        3.372200e+06   
4 -1.851283  1.071885 -0.411990  0.178246       ...        2.459121e+06   

       Force_14        EoM_14    CCI_14  KelChM_14  KelChU_14  KelChD_14  \
0 -2.921157e+07  2.286739e-07  0.959960  93.032764  95.832764  90.232765   
1  5.695471e+06  2.086708e-07  0.713277  93.570735  96.351449  90.790020   
2 -4.066156e+07  2.500726e-07  0.779588  94.192408  96.916693  91.468123   
3 -4.964177e+07  2.263920e-07  0.762796  94.770295  97.348152  92.192438   
4 -1.593928e+07  2.007584e-07  0.060704  95.249347  97.800775  92.697918   

   Ultimate_Osc  Donchian_14  Future Returns  
0      0.034641    13.389999        0.178246  
1      0.031385    13.389999       -0.411990  
2      0.058735    13.389999        1.071885  
3      0.057835    13.389999       -1.851283  
4      0.026201    13.389999        1.213226  

[5 rows x 50 columns]

In [49]:
# ### Do not need to drop. Just don't use tomorrows data to predict tomorrows prices.
# ### Hence, only use todays data to predict tomorrows prices
# # drop unneded columns
#df_features.drop(['Open','Close','Low','High','Today'], axis = 1, inplace = True)

df_features = df_features.set_index(['Date'])

# stores all labels
y = df_features['Future Returns']

# drop labels
df_features.drop(['Future Returns'], axis = 1, inplace = True)

# # all training data
X_train_all = df_features.loc[df_features.index < '2018-01-01']
y_train_all = y[X_train_all.index]

# # creates all test data which is all after January 2018
X_test = df_features.loc[(df_features.index >= '2018-01-01'),:]
y_test = y[X_test.index]

In [50]:
print(X_train_all.shape)

(3979, 48)


In [51]:
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import LinearRegression

predict = []
actual = []
for count in range(1,500):
    X_train_i = X_train_all.iloc[0:(3454+count-1), :]
    y_train_i = y_train_all.iloc[0:(3454+count-1)]
     
    X_val_i = X_train_all.iloc[(3454+count),:].reshape(1,-1)
    y_val_i = y_train_all.iloc[(3454+count)]
        
#     rf = RandomForestRegressor(random_state = 0)
#     rf.fit(X_train_i, y_train_i)
    model = BayesianRidge(fit_intercept = True)
    model.fit(X_train_i, y_train_i)
    
    predict.append(model.predict(X_val_i)[0])
    actual.append(y_val_i)        

AttributeError: 'Series' object has no attribute 'reshape'

In [53]:
mean_squared_error(actual, predict)

In [45]:
r2_score(actual, predict)

-0.004875944857932746

In [33]:
print(r2_score(y_test, model.predict(X_test)))

-0.022087377128748642


### Train-Validation Split & Train Models

##### Run Models
- Random Forest
- Bagging
- Boosting
- Linear Regression

##### NOTE
- Need to retrain the model for each prediction. Suppose 200 Days total.
- Hence,
   - Train_1[start date, 100] --> Predict y[101]
   - Train_2[start date, 101] --> Predict y[102]
   - Train_3[start date, 103] --> Predict y[103]
   - ...
   - Train_199[start date, 198] --> Predict y[199]
   - Train_200[start date, 199] --> Predict y[200]
- Thus, to forecast 200 daily returns we need to train 200 different models!!!
   


#### Jonathans Code

In [100]:
# do time series split DOING NUMBER 2
splits = TimeSeriesSplit(n_splits=5)

# Initialize empty arrays
trainsamples  = [] # [664, 1327, 1990, 2653, 3316] # sizes of training samples
model1metrics = [] # Random Forest
model2metrics = [] # Bagging
model3metrics = [] # Boosting
model4metrics = [] # Linear Regression


for train_index, test_index in splits.split(X_train_all.loc[:,'Date']):
    
    # do the split for train, Take row 0 to last element of train index
    X_train = X_train_all.loc[0:train_index[len(train_index) - 1],:].copy(deep=True)
    y_train = y[X_train.index]
        
    # do the split for validation, Take first element of test index to last element of test index
    X_val = X_train_all.loc[test_index[0]:test_index[len(test_index) - 1],:].copy(deep=True)
    y_val = y[X_val.index]
        
    # Print Statements
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print('Observations: ', (X_train.shape[0] + X_test.shape[0]))
    print('Cutoff date, or first date in validation data: ', X_val.iloc[0,0])
    print('Training Observations: ', (X_train.shape[0]))
    print('Testing Observations: ', (X_test.shape[0]))
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    
    # drop date
    X_train.drop(['Date'], axis = 1, inplace = True)
    X_val.drop(['Date'], axis = 1, inplace = True)
    
    # Append current size of training sample
    trainsamples.append(X_train.shape[0]) # .shape[0] --> num rows
    
### Random Forest
    # random forest regression based on default parameter
    rf = RandomForestRegressor(random_state = 0)
    rf.fit(X_train, y_train)
    model1metrics.append([mean_squared_error(y_val, rf.predict(X_val)),r2_score(y_val, rf.predict(X_val))])
    
# ### Bagging
#     # bagging based on default parameter
#     bag = BaggingRegressor(DecisionTreeRegressor())
#     bag.fit(X_train, y_train)
#     model2metrics.append([mean_squared_error(y_val, bag.predict(X_val)),r2_score(y_val, bag.predict(X_val))])

# ### Boosting
#     # boosting on default parameter
#     boost = AdaBoostRegressor(DecisionTreeRegressor(), random_state = 0, learning_rate=0.01)
#     boost.fit(X_train, y_train)
#     model3metrics.append([mean_squared_error(y_val, boost.predict(X_val)),r2_score(y_val, boost.predict(X_val))])
    
# ### Linear Regression
#     # linear regression on default parameter
#     lr = linear_model.LinearRegression()
#     lr.fit(X_train, y_train)
#     model4metrics.append([mean_squared_error(y_val, lr.predict(X_val)),r2_score(y_val, lr.predict(X_val))])

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Observations:  875
Cutoff date, or first date in validation data:  2004-11-01 00:00:00
Training Observations:  664
Testing Observations:  211
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


/Users/YoungFreeesh/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Observations:  1538
Cutoff date, or first date in validation data:  2007-06-21 00:00:00
Training Observations:  1327
Testing Observations:  211
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


/Users/YoungFreeesh/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Observations:  2201
Cutoff date, or first date in validation data:  2010-02-08 00:00:00
Training Observations:  1990
Testing Observations:  211
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


/Users/YoungFreeesh/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Observations:  2864
Cutoff date, or first date in validation data:  2012-09-24 00:00:00
Training Observations:  2653
Testing Observations:  211
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


/Users/YoungFreeesh/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Observations:  3527
Cutoff date, or first date in validation data:  2015-05-15 00:00:00
Training Observations:  3316
Testing Observations:  211
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


/Users/YoungFreeesh/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [110]:
model1metrics

[[0.0045937039913561935, 0.9954591784110058],
 [0.07477435108321685, 0.9794299412602301],
 [0.0004523796920241774, 0.9996845945462203],
 [0.0012142172454971058, 0.9990749091061193],
 [0.000794832356181995, 0.9994342788419209]]

#### Sams Code

In [118]:

### Random Forest - Train-Validate model
# in this case we can consider validation set as a test set

# Set the number of predictions we want to make
# Note - Num Predictions == Num models we'll need to train
trainSize = 3316 # chosen arbitrarily
numPreds = X_train_all.shape[0] - trainSize # Number of Predictions = numRows - training size

# for each prediction # 662 predictions
for i in range(numPreds): # i = 0,1,2,...,661
    trainSize += i
    # set temp train/test sets
    
    # do the split for train, Take row 0 to last element of train index
    X_train = X_train_all.loc[0:trainSize,:]
    y_train = y[X_train.index]
    





In [106]:
X_val.head()

High         Low        Open   Volume       Close     Today  \
3316  174.410004  172.600006  173.910004  2916600  149.597687 -0.453896   
3317  173.490005  172.300003  173.440002  1923600  149.424957 -0.115463   
3318  173.750000  171.929993  172.970001  2523000  149.787643  0.242721   
3319  174.440002  172.460007  173.330002  2300300  150.029404  0.161402   
3320  174.139999  173.039993  173.320007  2295600  149.666763 -0.241713   

          lag1      lag2      lag3      lag4     ...         MFI_14  \
3316  1.027401  1.014373 -0.333112 -0.903399     ...       0.571429   
3317 -0.453896  1.027401  1.014373 -0.333112     ...       0.500000   
3318 -0.115463 -0.453896  1.027401  1.014373     ...       0.500000   
3319  0.242721 -0.115463 -0.453896  1.027401     ...       0.571429   
3320  0.161402  0.242721 -0.115463 -0.453896     ...       0.500000   

             OBV_14      Force_14        EoM_14    CCI_14   KelChM_14  \
3316  523378.571429 -1.111363e+06 -1.267987e-07  1.040867  164.342709   
3317  -29500.000000 -5.935885e+06 -2.374802e-07  0.574141  164.404726   
3318 -172028.571429 -2.951631e+06 -2.996166e-07  0.671242  164.386374   
3319  353914.285714 -1.206062e+07 -1.680592e-07  0.949665  164.545557   
3320  -46635.714286 -2.011766e+06 -1.946159e-07  0.844002  164.610773   

       KelChU_14   KelChD_14  Ultimate_Osc  Donchian_14  
3316  166.943423  161.741994      0.129086    14.660004  
3317  166.756869  162.052583      0.065950    15.100006  
3318  166.702089  162.070658      0.040401    15.100006  
3319  166.718415  162.372698      0.093207    15.100006  
3320  166.749346  162.472201      0.082987    16.270004  

[5 rows x 48 columns]

In [107]:
y_val.head()
rf.predict(X_val).head()

3316   -0.453896
3317   -0.115463
3318    0.242721
3319    0.161402
3320   -0.241713
Name: Today, dtype: float64

In [112]:
X_train_all
X_train_all.shape[0]

3979

In [117]:
train_index[len(train_index) - 1]

3315